# Singapore University Acceptance Classification Model


In [1]:
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load Data

In [2]:
df = pd.read_csv("data/2023-NUS.csv", header=1)
df.sample(10)

,Nationality,Education Track,Results (GPA),A-level Results,IB Score,University,Course Name,Results
187,Singaporean,JC,NaN,86.25,NaN,NUS,Information System,Accepted by direct entry/discovery day
185,Singaporean,JC,NaN,82.5,NaN,NUS,Information Security,Accepted by direct entry/discovery day
167,Singaporean,JC,NaN,73.125+2.5fcb,NaN,NUS,Engineering With Preferred Major In Mechanical...,Accepted by direct entry/discovery day
58,Singaporean,JC,NaN,82.5 (without first choice bonus),NaN,NUS,College Of Humanities And Sciences,Rejected
228,Singaporean,JC,NaN,90 + H3 Bio DIST + H3 Chem DIST,NaN,NUS,Pharmaceutical Science,Accepted by direct entry/discovery day
233,Singaporean,JC,NaN,88.75,NaN,NUS,Pharmacy,Accepted by direct entry/discovery day
132,Singaporean,JC,NaN,90 + H3 Pass,NaN,NUS,Computer Science With Second Major In Mathematics,Accepted by direct entry/discovery day
216,Singaporean,JC,NaN,87.5,NaN,NUS,Medicine,Accepted by direct entry/discovery day
77,Singaporean,JC,NaN,86.25,NaN,NUS,College Of Humanities And Sciences (Psychology),Accepted by direct entry/discovery day
19,Singaporean,JC,NaN,87.5,NaN,NUS,Business Administration And Computer Science,Rejected


## Preprocessing

In [3]:
# Rename columns
df.columns = [
    "nationality",
    "education",
    "gpa",
    "alevel",
    "ibscore",
    "university",
    "course",
    "accepted",
]

# Change column data to lowercase.
columns_to_lower = ["nationality", "education", "course", "accepted"]
for column in columns_to_lower:
    df[column] = df[column].str.lower()

# Applying one-hot encoding to 'nationality' and 'education
df = pd.get_dummies(df, columns=["nationality", "education"])

# Encode results with 0 or 1.
df["accepted"] = df["accepted"].apply(lambda x: True if "accepted" in x else False)

# Drop 'university' for now because all NUS.
df = df.drop("university", axis=1)

# Also drop 'course' for now.
df = df.drop("course", axis=1)

# Remove alphabetical characters from alevel column.
pattern = r"\d{2}.?\d*"
df["alevel"] = df["alevel"].apply(
    lambda x: re.search(pattern, x).group() if isinstance(x, str) else np.nan
)

display(df)

,gpa,alevel,ibscore,accepted,nationality_international,nationality_pr,nationality_singaporean,education_ib,education_jc,education_poly
0,3.94,NaN,NaN,True,False,False,True,False,False,True
1,3.00,NaN,NaN,False,False,False,True,False,False,True
2,3.69,NaN,NaN,True,False,False,True,False,False,True
3,NaN,76.25,NaN,True,False,False,True,False,True,False
4,NaN,78.75,NaN,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...
240,NaN,87.5,NaN,False,False,False,True,False,True,False
241,NaN,78.75,NaN,True,False,False,True,False,True,False
242,NaN,75,NaN,True,False,False,True,False,True,False
243,NaN,83.75,NaN,True,False,False,True,False,True,False


### Scaling Grades

In [4]:
grade_columns = ["gpa", "alevel", "ibscore"]
grade_scalers = []

for column in grade_columns:
    scaler = StandardScaler()
    df[column] = scaler.fit_transform(df[column].to_numpy().reshape(-1, 1))
    grade_scalers.append(scaler)

display(df[grade_columns])

,gpa,alevel,ibscore
0,0.791853,NaN,NaN
1,-3.972073,NaN,NaN
2,-0.475149,NaN,NaN
3,NaN,-1.681346,NaN
4,NaN,-1.190893,NaN
...,...,...,...
240,NaN,0.525692,NaN
241,NaN,-1.190893,NaN
242,NaN,-1.926573,NaN
243,NaN,-0.209987,NaN


### Merging Grades

After scaling `gpa`, `alevel`, `ibscore`, merge them into a single `grades` column.

In [5]:
df["grades"] = df["alevel"].fillna(df["gpa"]).fillna(df["ibscore"])
df = df.drop(grade_columns, axis=1)
display(df)

,accepted,nationality_international,nationality_pr,nationality_singaporean,education_ib,education_jc,education_poly,grades
0,True,False,False,True,False,False,True,0.791853
1,False,False,False,True,False,False,True,-3.972073
2,True,False,False,True,False,False,True,-0.475149
3,True,False,False,True,False,True,False,-1.681346
4,False,False,False,True,False,True,False,-1.190893
...,...,...,...,...,...,...,...,...
240,False,False,False,True,False,True,False,0.525692
241,True,False,False,True,False,True,False,-1.190893
242,True,False,False,True,False,True,False,-1.926573
243,True,False,False,True,False,True,False,-0.209987


### Splitting Data

In [6]:
X = df.drop("accepted", axis=1)
y = df["accepted"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(196, 7) (196,)
(49, 7) (49,)


## Classification Models

### Logistic Regression

In [7]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

print(f"Train Score: {accuracy_score(train_pred, y_train)}")
print(f"Test Score: {accuracy_score(test_pred, y_test)}")

Train Score: 0.8673469387755102
Test Score: 0.8163265306122449
